[medium](https://medium.com/@jonathan_hui/rl-dqn-deep-q-network-e207751f7ae4) <br>
[github](https://github.com/udacity/deep-reinforcement-learning/tree/master/dqn) <br>
[paper](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf)

In [1]:
%reload_ext autoreload
%autoreload 2
import torch
import torch.nn.functional as F 
import random
import numpy as np
from EXITrl.approx_v_base import ApproxVBase
from EXITrl.approx_policy_base import ApproxPolicyBase
from EXITrl.base import Base
from EXITrl.helpers import update_params, ExperienceReplay, WeightDecay, device
from EXITrl.nn_wrapper import NNWrapper
import gym

In [2]:
class QNetwork(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.linear1 = torch.nn.Linear(input_size, hidden_size)
        self.linear2 = torch.nn.Linear(hidden_size, hidden_size)
        self.linear3 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        return self.linear3(x)

In [8]:
class DQN(Base):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.local_q_network = NNWrapper(
            QNetwork(self.num_state, 64, self.num_action),
            lr=self.alpha
        )
        self.target_q_network = NNWrapper(
            QNetwork(self.num_state, 64, self.num_action),
            lr=0 # use manual update
        )
        update_params(self.local_q_network.model, self.target_q_network.model, tau=0)
        self.num_step = 0
    
    def initialize(self, num_step_to_learn, eps_start, eps_end, eps_decay, num_experience, num_recall, skip_frame=1):
        self.num_step_to_learn= num_step_to_learn
        self.epsilon_decay = WeightDecay(eps_start, eps_end, eps_decay)
        self.epsilon = self.epsilon_decay.step()
        self.experience_replay = ExperienceReplay(num_experience=num_experience, num_recall=num_recall)
        self.skip_frame = skip_frame
        
    def policy(self, state):
        return self.local_q_network.epsilon_greedy(state, self.epsilon)
    
    def learn(self, state, action, reward, next_state, done):
        # detach because we only backprop local network and update target network weight manually
        targets_next_Q = self.target_q_network.forward(next_state).detach().max(1)[0]
        targets_Q = reward + (self.gamma * targets_next_Q * (1 - done))

        local_Q = self.local_q_network.forward(state)
        expected_Q = local_Q.gather(1, action.unsqueeze(1).long()).squeeze(1)

        loss = F.mse_loss(expected_Q, targets_Q)
        self.local_q_network.backprop(loss)

        update_params(self.local_q_network.model, self.target_q_network.model, self.tau)

    def _loop(self, episode) -> int:
        total_reward = 0
        state = self.env.reset()
        for i in range(1000):
            action = self.policy(state)
            for _ in range(self.skip_frame):
                _state, reward, done, _ = self.env.step(action)
                if done: break
            self.experience_replay.remember(state, action, reward, _state, done)
            
            self.num_step += 1
            if self.num_step%self.num_step_to_learn == 0:
                experiences = self.experience_replay.recall()
                self.learn(*experiences)
            state = _state
            
            total_reward += reward
            self.additional_log['num_step'] = self.num_step
            if done: break
        self.epsilon = self.epsilon_decay.step()
        return total_reward
    
    def _save(self, reward):
        torch.save(self.local_q_network.model.state_dict(), self.save_name)
    def _load(self):
        self.epsilon = 0
        self.local_q_network.model.load_state_dict(torch.load(self.save_name, map_location=device))
        self.local_q_network.model.eval()


### LunarLander

In [9]:
try: env.close()
except: pass
env = gym.make('LunarLander-v2')
dqn = DQN(env, 
      num_mean_episode=100,
      num_episodes=2000,
      alpha=5e-4, 
      gamma=.99,
      tau=1e-3,
      save_name="checkpoint/LunarLander-v2-DQN.pth")
dqn.initialize(num_step_to_learn=4, 
               eps_start=1, 
               eps_end=.01, 
               eps_decay=.995, 
               num_experience=2048, 
               num_recall=512,
               skip_frame=1)
dqn.train(early_stop=lambda mean_reward: mean_reward>200)
# dqn.play()

Episode 100	Average Score: -158.27 	other{'num_step': 11629}
Episode 200	Average Score: -111.80 	other{'num_step': 32919}
Episode 300	Average Score: -57.77 	other{'num_step': 83802}}
Episode 400	Average Score: -34.82 	other{'num_step': 153762}
Episode 500	Average Score: 66.82 	other{'num_step': 211704}}
Episode 600	Average Score: 153.99 	other{'num_step': 250725}
Episode 700	Average Score: 128.93 	other{'num_step': 292653}
Episode 800	Average Score: 176.08 	other{'num_step': 332247}
Episode 900	Average Score: 190.32 	other{'num_step': 368841}
--- early stop ---- Score: 200.77 	other{'num_step': 370030}
 current_mean_reward: 200.77107305470724 episode: 904


### Skip frame

In [143]:
try: env.close()
except: pass
env = gym.make('LunarLander-v2')
dqn = DQN(env, 
      num_mean_episode=100,
      num_episodes=2000,
      alpha=5e-4, 
      gamma=.95,
      tau=1e-3,
      save_name="checkpoint/LunarLander-v2-DQN.pth")
dqn.initialize(num_step_to_learn=4, 
               eps_start=1, 
               eps_end=.01, 
               eps_decay=.995, 
               num_experience=2048, 
               num_recall=32,
               skip_frame=3)
dqn.train(early_stop=lambda mean_reward: mean_reward>200)
# dqn.play()

Episode 100	Average Score: -148.99
Episode 200	Average Score: -142.11
Episode 300	Average Score: -149.07
Episode 400	Average Score: -150.33
Episode 500	Average Score: -166.24
Episode 600	Average Score: -175.00
Episode 700	Average Score: -155.82
Episode 800	Average Score: -142.49
Episode 900	Average Score: -95.593
Episode 1000	Average Score: -10.25
Episode 1100	Average Score: 32.34
Episode 1200	Average Score: 36.93
Episode 1300	Average Score: 31.62
Episode 1400	Average Score: 53.91
Episode 1500	Average Score: 56.46
Episode 1600	Average Score: 70.57
Episode 1700	Average Score: 54.78
Episode 1800	Average Score: 103.50
Episode 1900	Average Score: 102.85
Episode 2000	Average Score: 85.092


### Breakout

In [4]:
try: env.close()
except: pass
env = gym.make('Breakout-ram-v0')
dqn = DQN(env, 
      num_mean_episode=100,
      num_episodes=int(1e6),
      alpha=5e-4, 
      gamma=.95,
      tau=1e-3,
      save_name="checkpoint/Breakout-ram-v0-DQN.pth")
dqn.initialize(num_step_to_learn=4, 
               eps_start=1, 
               eps_end=.01, 
               eps_decay=.9999, 
               num_experience=20480, 
               num_recall=1024,
               skip_frame=1)
dqn.train(early_stop=lambda mean_reward: mean_reward>200)
# dqn.play()

Episode 100	Average Score: 1.42
Episode 200	Average Score: 1.44
Episode 300	Average Score: 1.37
Episode 400	Average Score: 1.14
Episode 500	Average Score: 1.13
Episode 600	Average Score: 1.43
Episode 700	Average Score: 1.45
Episode 800	Average Score: 1.41
Episode 900	Average Score: 1.77
Episode 1000	Average Score: 1.87
Episode 1100	Average Score: 1.69
Episode 1200	Average Score: 1.84
Episode 1300	Average Score: 1.78
Episode 1400	Average Score: 1.92
Episode 1500	Average Score: 1.76
Episode 1600	Average Score: 1.93
Episode 1700	Average Score: 2.06
Episode 1800	Average Score: 2.27
Episode 1900	Average Score: 2.04
Episode 2000	Average Score: 1.83
Episode 2100	Average Score: 2.35
Episode 2200	Average Score: 2.24
Episode 2300	Average Score: 2.30
Episode 2400	Average Score: 2.50
Episode 2500	Average Score: 2.84
Episode 2600	Average Score: 3.05
Episode 2700	Average Score: 3.50
Episode 2800	Average Score: 3.26
Episode 2900	Average Score: 3.66
Episode 3000	Average Score: 3.46
Episode 3100	Averag

KeyboardInterrupt: 

### Pong

In [145]:
try: env.close()
except: pass
env = gym.make('Pong-ram-v0')
dqn = DQN(env, 
      num_mean_episode=100,
      num_episodes=int(1e6),
      alpha=5e-4, 
      gamma=.99,
      tau=1e-3,
      save_name="checkpoint/Pong-ram-v0-DQN.pth")
dqn.initialize(num_step_to_learn=4, 
               eps_start=1, 
               eps_end=.01, 
               eps_decay=.995, 
               num_experience=2048, 
               num_recall=64,
               skip_frame=2)
dqn.train(early_stop=lambda mean_reward: mean_reward>200)
# dqn.play()

Episode 100	Average Score: -10.37
Episode 200	Average Score: -10.57
Episode 300	Average Score: -10.79
Episode 366	Average Score: -10.72

KeyboardInterrupt: 